In [1]:
from fastai.text import *
import numpy as np
from sklearn.model_selection import train_test_split
import pickle
import sentencepiece as spm
import re
import pdb

In [2]:
import fastai, torch
fastai.__version__ , torch.__version__

('1.0.61', '1.0.0')

In [3]:
torch.cuda.set_device(0)

In [4]:
def random_seed(seed_value, use_cuda):
    np.random.seed(seed_value)  
    torch.manual_seed(seed_value)  
    random.seed(seed_value)
    if use_cuda:
        torch.cuda.manual_seed(seed_value)
        torch.cuda.manual_seed_all(seed_value)  
        torch.backends.cudnn.deterministic = True
        torch.backends.cudnn.benchmark = False

In [5]:
random_seed(42, True)

In [6]:
path = Path('./')

In [7]:
!pwd

/data/home/ubuntu/gaurav/in/nlp-for-hindi/classification-bbc-news


In [8]:
df_train = pd.read_csv(path/'../../classification_public_datasets/iitp+-movie-reviews/hi/hi-train.csv', header=None)
df_train.head()

,0,1
0,neutral,"निर्माता :\nशीतल विनोद तलवार, मधु‍ मैंटेना\n\n..."
1,positive,’उड़ान’ से विक्रमादित्य\nमोटवाने\nने अच्छे सिन...
2,neutral,फिल्म में गानों के दृश्य में अनुष्का को माइक क...
3,neutral,फांसी चढ़ने से पहले वह पाकिस्तान के सदर से खास...
4,positive,राज कुमार हिरानी ने इस मूवी में भी अपने डायरेक...


In [9]:
df_valid = pd.read_csv(path/'../../classification_public_datasets/iitp+-movie-reviews/hi/hi-valid.csv', header=None)
df_valid.head()

,0,1
0,negative,"ये फगली-फगली क्या है, ये फगली-फगली? इस बात का ..."
1,neutral,निर्माता :\nआदित्य चोपड़ा\n\nनिर्देशक :\nहबीब ...
2,positive,इस कसौटी पर करण काफी हद तक खरे भी उतरे हैं।
3,positive,"छोटे शहर की गालियां, छतें और विहंगम दृश्य मोहक..."
4,positive,मनीष और जिशान का काम भी लोगों को याद रहेगा।


In [10]:
df_test = pd.read_csv(path/'../../classification_public_datasets/iitp+-movie-reviews/hi/hi-test.csv', header=None)
df_test.head()

,0,1
0,negative,काव्या अपनी खुन्नस और जिद में सिद्धार्थ को तबा...
1,negative,"निर्माता :\nफरहान अख्तर, रितेश सिधवानी\n\nनिर्..."
2,positive,असमर्पित रिश्ते में यकीन रखने वाले दोनों मौज-म...
3,neutral,"बेवकूफियां को हबीब फैज़ल ने लिखा है, जिनका नाम..."
4,neutral,तब वह मोटी थी और जेरी ने उस पर ध्यान नहीं दिय...


In [11]:
df_train.shape, df_valid.shape, df_test.shape

((2480, 2), (310, 2), (310, 2))

In [12]:
df_train[df_train[1].isnull()]

,0,1


In [13]:
# df_train = df_train.dropna()

In [14]:
df_test[df_test[1].isnull()]

,0,1


In [15]:
# df_test = df_test.dropna()

In [16]:
label_cols = [0]

In [17]:
class HindiTokenizer(BaseTokenizer):
    def __init__(self, lang:str):
        self.lang = lang
        self.sp = spm.SentencePieceProcessor()
        self.sp.Load(str('./../../models/hindi/tokenizer/Created from Hindi Wikipedia Articles - 172k/hindi_lm_large.model'))
        
    def tokenizer(self, t:str) -> List[str]:
        return self.sp.EncodeAsPieces(t)

In [18]:
sp = spm.SentencePieceProcessor()
sp.Load('./../../models/hindi/tokenizer/Created from Hindi Wikipedia Articles - 172k/hindi_lm_large.model')
itos = [sp.IdToPiece(int(i)) for i in range(30000)]

In [19]:
itos[:10]

['<unk>', '<s>', '</s>', '▁के', '।', '▁में', '▁है', ',', '▁की', '▁']

In [20]:
# 30,000 is the vocab size that we chose in sentencepiece
hindi_vocab = Vocab(itos)

In [21]:
tokenizer = Tokenizer(tok_func=HindiTokenizer, lang='hi')

In [22]:
tokenizer.special_cases

['xxunk',
 'xxpad',
 'xxbos',
 'xxeos',
 'xxfld',
 'xxmaj',
 'xxup',
 'xxrep',
 'xxwrep']

In [23]:
data_lm = TextLMDataBunch.from_df(path=path, train_df=df_train, valid_df=df_valid, test_df=df_test, tokenizer=tokenizer, vocab=hindi_vocab, bs=32)

In [24]:
data_lm.show_batch()

idx,text
0,▁स्पष्टीकरण ▁लिखा ▁हुआ ▁आता ▁है ▁कि ▁सारे ▁पात्र ▁और ▁कहानी ▁का ल्प िन क ▁हैं । ▁अगले ▁ही ▁पल ▁दिखाई ▁देता ▁है ▁कि ▁फिल्म ▁एक ▁सच्ची ▁कहानी ▁पर ▁आधारित ▁है । ▁शायद ▁कानूनी ▁उलझन ों ▁से ▁बचने ▁के ▁लिए ▁राम गोपाल ▁वर्मा ▁को ▁विरोधाभास ▁बातें ▁साथ ▁करनी ▁पड़ी ▁हो । ▁‘ रक्त चरित्र ’ ▁की ▁कहानी ▁आंध्रप्रदेश ▁के ▁रवि ▁पर िता ला ▁और ▁उसके ▁विरोधी ▁सूरी ▁पर ▁आधारित ▁है ।
1,"- ▁इस ▁फिल्म ▁की ▁रिलीज ▁से ▁कई ▁हफ्ते ▁पहले ▁ही ▁फिल्म ▁के ▁दो ▁गाने ▁' सू ट - सू ट ▁जि ंद ड़ी ' ▁और ▁' इश ्क ▁तेरा ▁त ड़ पा वे ' ▁म्यूजिक ▁लव र्स ▁की ▁जुबान ▁पर ▁हैं । ▁वहीं , ▁डायरेक्टर ▁साकेत ▁की ▁तारीफ ▁करनी ▁होगी ▁कि ▁उन्होंने ▁इन ▁गानों ▁को ▁ऐसी ▁सि चु ए शन ▁पर ▁फिट ▁किया ▁है ▁जहां ▁यह ▁कहानी ▁में ▁रुकावट ▁नहीं"
2,"▁नहीं ▁है ▁कि ▁यह ▁अच्छी ▁फिल्म ▁नहीं ▁है । ▁कुछ ▁कम ियों ▁के ▁बावजूद ▁ये ▁फिल्म ▁मनोरंजन ▁करने ▁में ▁कामयाब ▁होती ▁है । ▁बैनर ▁: ▁प्री ती श ▁नंदी ▁कम्युनिकेशन ्स , ▁बालाजी ▁मोशन ▁पिक्चर्स ▁निर्माता ▁: ▁रंग िता ▁प्री ती श ▁नंदी , ▁प्री ती श ▁नंदी , ▁एकता ▁कपूर ▁निर्देशक ▁: ▁साकेत ▁चौधरी ▁संगीत ▁: ▁प्रीतम ▁चक्रवर्ती ▁कलाकार ▁: ▁फरहान ▁अख्तर , ▁विद्या ▁बाल न , ▁राम ▁कपूर"
3,▁भारतीय ▁नागरिकों ▁के ▁साथ ▁रंजीत ▁का ▁परिवार ▁भी ▁फंस ▁जाता ▁है । ▁सरकारी ▁अधिकारी ▁हालात ▁से ▁निपटने ▁के ▁विकल्पों ▁पर ▁विचार ▁कर ▁रहे ▁हैं ▁और ▁ऐसे ▁में ▁कुवैत ▁का ▁रईस ▁बि ज़न स मैन ▁रंजीत ▁अपने ▁प्रभाव ▁और ▁रस ू ख ▁से ▁अपनी ▁फैमिली ▁और ▁फंसे ▁दूसरे ▁भारतीय ▁नागरिकों ▁को ▁वहां ▁से ▁निकालने ▁का ▁मिशन ▁स्टार्ट ▁करता ▁है । ▁कुवैत ▁में ▁हालात ▁खराब ▁हो ▁रहे ▁थे । ▁भारतीय ▁सेना ▁डायरेक्ट
4,"▁हिंदी ▁में ▁' श ॉर्ट कट ▁रोम िया ' ▁टा य टल ▁के ▁साथ ▁पेश ▁किया ▁है । ▁x x bo s ▁निर्माता ▁: ▁नीरज ▁पाठक , ▁कृष्ण ▁चौधरी ▁निर्देशक ▁: ▁नीरज ▁पाठक ▁संगीत ▁: ▁मोंट ी ▁शर्मा ▁कलाकार ▁: ▁सन ी ▁देओल , ▁ईशा ▁को प्प िक र , ▁इर फान ▁खान , ▁कोंकण ा ▁से न ▁शर्मा , ▁दी पल ▁शॉ , ▁गोविंद ▁नामदेव ▁यू ▁/ ▁ए ▁*"


In [25]:
learn = language_model_learner(data_lm, AWD_LSTM, drop_mult=0.3)

In [26]:
# Loading the pretrained language model on hindi wikipedia
learn.load('../../../models/hindi/lm/ULMFiT - Trained on Hindi Wikipedia Articles - 172k/model')

LanguageLearner(data=TextLMDataBunch;

Train: LabelList (2480 items)
x: LMTextList
▁x x bo s ▁निर्माता ▁: ▁शीतल ▁विनोद ▁तलवार , ▁मधु ▁मैं टे ना ▁निर्देशक ▁: ▁राम गोपाल ▁वर्मा ▁कलाकार ▁: ▁विवेक ▁ओबेरॉय , ▁शत्रुघ्न ▁सिन्हा , ▁अभिमन्यु ▁सिंह , ▁सु शांत ▁सिंह , ▁जरी न ▁वह ाब , ▁आशीष ▁विद्यार्थी , ▁राजा ▁कृष्ण मूर्ति , ▁श्रीनिवास ▁राव ▁* ▁केवल ▁वयस्कों ▁के ▁लिए ▁* ▁16 ▁री ल ▁* ▁2 ▁घंटे ▁14 ▁मिनट ▁’ रक्त चरित्र -1 ’ ▁शुरू ▁होते ▁ही ▁स्पष्टीकरण ▁लिखा ▁हुआ ▁आता ▁है ▁कि ▁सारे ▁पात्र ▁और ▁कहानी ▁का ल्प िन क ▁हैं । ▁अगले ▁ही ▁पल ▁दिखाई ▁देता ▁है ▁कि ▁फिल्म ▁एक ▁सच्ची ▁कहानी ▁पर ▁आधारित ▁है । ▁शायद ▁कानूनी ▁उलझन ों ▁से ▁बचने ▁के ▁लिए ▁राम गोपाल ▁वर्मा ▁को ▁विरोधाभास ▁बातें ▁साथ ▁करनी ▁पड़ी ▁हो । ▁‘ रक्त चरित्र ’ ▁की ▁कहानी ▁आंध्रप्रदेश ▁के ▁रवि ▁पर िता ला ▁और ▁उसके ▁विरोधी ▁सूरी ▁पर ▁आधारित ▁है । ▁राम ू ▁ने ▁उससे ▁प्रेरणा ▁लेकर , ▁कुछ ▁अपनी ▁कल्पना ▁मिलाकर ▁‘ रक्त चरित्र ’ ▁को ▁दो ▁भागों ▁में ▁बनाया ▁है । ▁पहले ▁भाग ▁में ▁दिखाया ▁गया ▁है ▁कि ▁किस ▁तरह ▁अपने ▁पिता ▁और ▁भाई ▁की ▁हत्या ▁होने ▁के ▁बाद

In [27]:
# Fine tuning the prtrained LM on current dataset

In [28]:
learn.lr_find()

epoch,train_loss,valid_loss,accuracy,time


LR Finder is complete, type {learner_name}.recorder.plot() to see the graph.


In [29]:
learn.recorder.plot()

In [30]:
learn.freeze()

In [31]:
learn.fit_one_cycle(1, 1e-2, moms=(0.8,0.7))

epoch,train_loss,valid_loss,accuracy,time
0,3.939221,3.823890,0.348142,00:17


In [32]:
learn.save('fit_head', with_opt=True)

In [33]:
learn.load('fit_head', with_opt=True);

In [34]:
learn.unfreeze()

In [35]:
learn.fit_one_cycle(3, 1e-3, moms=(0.8,0.7))

epoch,train_loss,valid_loss,accuracy,time
0,3.429090,3.664991,0.367845,00:21
1,3.182533,3.560144,0.382553,00:21
2,2.869652,3.567188,0.384001,00:21


In [36]:
learn.save('fine_tuned', with_opt=True)

In [37]:
learn.load('fine_tuned', with_opt=True);

In [38]:
learn.predict('▁इस ▁लाइन ▁के ▁चलने ▁से ▁दक्षिणी ',n_words=10)

'▁इस ▁लाइन ▁के ▁चलने ▁से ▁दक्षिणी  ▁दौर ▁में ▁ऐसा ▁लगता ▁है ▁कि ▁दक्षिण ▁भारत ▁में ▁इस'

In [39]:
learn.save_encoder('fine_tuned_enc')

In [40]:
data_clas = TextClasDataBunch.from_df(path=path, train_df=df_train, valid_df=df_valid, test_df=df_test, tokenizer=tokenizer, vocab=hindi_vocab)

In [41]:
data_clas.show_batch()

text,target
"▁x x bo s ▁फराह ▁खान ▁के ▁परिवार ▁के ▁सदस्यों ▁ <unk> फ राह ▁/ ▁साजिद ▁/ ▁शिर ी ष ) ▁का ▁नाम ▁पोस्टर ▁पर ▁देख ▁डर ▁लगता ▁है ▁क्योंकि ▁इनके ▁द्वारा ▁' जो कर ', ▁' हम शक ल्स ', ▁' तीस ▁मार ▁खां ', ▁' हि म्म त वाला ' ▁जैसी ▁फिल्में ▁बनाई ▁गई ▁हैं ▁जो ▁अधिकांश ▁लोगों ▁के ▁लिए ▁टॉ र्च र ▁से ▁कम ▁नहीं ▁थी ।",neutral
"▁x x bo s ▁के री ▁पै कर ▁का ▁नाम ▁क्रिकेट ▁की ▁दुनिया ▁में ▁भले ▁ही ▁आदर ▁के ▁साथ ▁न ▁लिया ▁जाता ▁हो , ▁लेकिन ▁इस ▁शख्स ▁ने ▁1977 ▁में ▁क्रिकेट ▁में ▁छिपे ▁व्यापार ▁की ▁संभावनाओं ▁को ▁तलाश ▁लिया ▁था । ▁उन्होंने ▁विश्व ▁के ▁नाम ी ▁खिलाड़ियों ▁को ▁मुंह मा ंगा ▁धन ▁देकर ▁ अनुबंधित ▁किया ▁और ▁वर्ल्ड ▁सी रिज ▁क्रिकेट ▁के ▁तहत ▁उन्हें ▁खिला या ▁गया । ▁इसे ▁के",positive
"▁x x bo s ▁राजकुमार ▁ही रानी ▁जैसे ▁ऊंचे ▁कद ▁के ▁निर्देशक ▁के ▁लिए ▁संजय ▁दत्त ▁जैसा ▁विषय ▁स्तर ▁से ▁नीचे ▁है । ▁आखिर ▁संजय ▁दत्त ▁के ▁जीवन ▁में ▁ऐसा ▁क्या ▁है ▁कि ▁ही रानी ▁को ▁उस ▁पर ▁फिल्म ▁बनाने ▁की ▁जरूरत ▁महसूस ▁हो । ▁नि : संदेह ▁संजय ▁के ▁जीवन ▁में ▁कई ▁ऐसे ▁उतार - चढ़ाव ▁हैं ▁जिस ▁पर ▁कमर्शियल ▁फिल्म ▁बनाई ▁जा ▁सकती ▁है , ▁लेकिन ▁ही रानी",positive
"▁x x bo s ▁द ंगल ▁को ▁सिर्फ ▁स्पोर्ट्स ▁फिल्म ▁कहना ▁गलत ▁होगा । ▁इस ▁फिल्म ▁में ▁कई ▁रंग ▁हैं । ▁लड़कियों ▁के ▁प्रति ▁समाज ▁की ▁सोच , ▁रू ढि ़ वादी ▁परंपराएं , ▁एक ▁व्यक्ति ▁का ▁सपना ▁और ▁जुनून , ▁लड़के ▁की ▁चाह , ▁अखाड़ े ▁और ▁अखाड़ े ▁से ▁बाहर ▁के ▁दांव पे ंच , ▁देश ▁के ▁लिए ▁कुछ ▁कर ▁गुजरने ▁की ▁तमन्ना , ▁चैम्पियन ▁बनने ▁के ▁लिए",positive
▁x x bo s ▁कई ▁बार ▁चीजें ▁इतनी ▁पर फ ेक्ट ▁हो ▁जाती ▁हैं ▁कि ▁यकीन ▁करना ▁पड़ता ▁है ▁कि ▁किसी ▁अदृश्य ▁शक्ति ▁का ▁साथ ▁था । ▁1975 ▁में ▁रिलीज ▁हुई ▁‘ शो ले ’ ▁को ▁बनाते ▁समय ▁किसी ▁ने ▁उम्मीद ▁नहीं ▁की ▁थी ▁कि ▁वे ▁सब ▁इतिहास ▁लिखने ▁जा ▁रहे ▁हैं । ▁एक ▁ऐसी ▁फिल्म ▁बनाने ▁जा ▁रहे ▁हैं ▁जिसे ▁क्ला ▁सिक ▁माना ▁जाएगा ▁या ▁मिलि ने यम ▁की,positive


In [42]:
learn = text_classifier_learner(data_clas, AWD_LSTM, drop_mult=0.5)

In [43]:
learn.load_encoder('fine_tuned_enc')

RNNLearner(data=TextClasDataBunch;

Train: LabelList (2480 items)
x: TextList
▁x x bo s ▁निर्माता ▁: ▁शीतल ▁विनोद ▁तलवार , ▁मधु ▁मैं टे ना ▁निर्देशक ▁: ▁राम गोपाल ▁वर्मा ▁कलाकार ▁: ▁विवेक ▁ओबेरॉय , ▁शत्रुघ्न ▁सिन्हा , ▁अभिमन्यु ▁सिंह , ▁सु शांत ▁सिंह , ▁जरी न ▁वह ाब , ▁आशीष ▁विद्यार्थी , ▁राजा ▁कृष्ण मूर्ति , ▁श्रीनिवास ▁राव ▁* ▁केवल ▁वयस्कों ▁के ▁लिए ▁* ▁16 ▁री ल ▁* ▁2 ▁घंटे ▁14 ▁मिनट ▁’ रक्त चरित्र -1 ’ ▁शुरू ▁होते ▁ही ▁स्पष्टीकरण ▁लिखा ▁हुआ ▁आता ▁है ▁कि ▁सारे ▁पात्र ▁और ▁कहानी ▁का ल्प िन क ▁हैं । ▁अगले ▁ही ▁पल ▁दिखाई ▁देता ▁है ▁कि ▁फिल्म ▁एक ▁सच्ची ▁कहानी ▁पर ▁आधारित ▁है । ▁शायद ▁कानूनी ▁उलझन ों ▁से ▁बचने ▁के ▁लिए ▁राम गोपाल ▁वर्मा ▁को ▁विरोधाभास ▁बातें ▁साथ ▁करनी ▁पड़ी ▁हो । ▁‘ रक्त चरित्र ’ ▁की ▁कहानी ▁आंध्रप्रदेश ▁के ▁रवि ▁पर िता ला ▁और ▁उसके ▁विरोधी ▁सूरी ▁पर ▁आधारित ▁है । ▁राम ू ▁ने ▁उससे ▁प्रेरणा ▁लेकर , ▁कुछ ▁अपनी ▁कल्पना ▁मिलाकर ▁‘ रक्त चरित्र ’ ▁को ▁दो ▁भागों ▁में ▁बनाया ▁है । ▁पहले ▁भाग ▁में ▁दिखाया ▁गया ▁है ▁कि ▁किस ▁तरह ▁अपने ▁पिता ▁और ▁भाई ▁की ▁हत्या ▁होने ▁के ▁बाद ▁रवि

In [44]:
learn.freeze()

In [45]:
learn.lr_find()

LR Finder is complete, type {learner_name}.recorder.plot() to see the graph.


In [46]:
learn.recorder.plot()

In [47]:
learn.loss_func.func

CrossEntropyLoss()

In [48]:
mcc = MatthewsCorreff()

In [49]:
learn.metrics = [mcc, accuracy]

In [50]:
learn.fit_one_cycle(1, 1e-1, moms=(0.8,0.7))

epoch,train_loss,valid_loss,matthews_correff,accuracy,time
0,1.011237,1.014075,0.186293,0.461290,00:07


In [51]:
learn.save('first-full')

In [52]:
learn.load('first-full');

In [53]:
learn.freeze_to(-2)
learn.fit_one_cycle(5, slice(1e-2/(2.6**4),1e-2), moms=(0.8,0.7))

epoch,train_loss,valid_loss,matthews_correff,accuracy,time
0,0.911602,0.958300,0.300951,0.538710,00:09
1,0.864734,0.908979,0.357208,0.580645,00:07
2,0.778813,0.889175,0.410969,0.609677,00:08
3,0.678604,0.840619,0.455108,0.641935,00:09
4,0.580214,0.843147,0.452706,0.641935,00:08


In [54]:
learn.save('second-full')

In [55]:
learn.load('second-full')

RNNLearner(data=TextClasDataBunch;

Train: LabelList (2480 items)
x: TextList
▁x x bo s ▁निर्माता ▁: ▁शीतल ▁विनोद ▁तलवार , ▁मधु ▁मैं टे ना ▁निर्देशक ▁: ▁राम गोपाल ▁वर्मा ▁कलाकार ▁: ▁विवेक ▁ओबेरॉय , ▁शत्रुघ्न ▁सिन्हा , ▁अभिमन्यु ▁सिंह , ▁सु शांत ▁सिंह , ▁जरी न ▁वह ाब , ▁आशीष ▁विद्यार्थी , ▁राजा ▁कृष्ण मूर्ति , ▁श्रीनिवास ▁राव ▁* ▁केवल ▁वयस्कों ▁के ▁लिए ▁* ▁16 ▁री ल ▁* ▁2 ▁घंटे ▁14 ▁मिनट ▁’ रक्त चरित्र -1 ’ ▁शुरू ▁होते ▁ही ▁स्पष्टीकरण ▁लिखा ▁हुआ ▁आता ▁है ▁कि ▁सारे ▁पात्र ▁और ▁कहानी ▁का ल्प िन क ▁हैं । ▁अगले ▁ही ▁पल ▁दिखाई ▁देता ▁है ▁कि ▁फिल्म ▁एक ▁सच्ची ▁कहानी ▁पर ▁आधारित ▁है । ▁शायद ▁कानूनी ▁उलझन ों ▁से ▁बचने ▁के ▁लिए ▁राम गोपाल ▁वर्मा ▁को ▁विरोधाभास ▁बातें ▁साथ ▁करनी ▁पड़ी ▁हो । ▁‘ रक्त चरित्र ’ ▁की ▁कहानी ▁आंध्रप्रदेश ▁के ▁रवि ▁पर िता ला ▁और ▁उसके ▁विरोधी ▁सूरी ▁पर ▁आधारित ▁है । ▁राम ू ▁ने ▁उससे ▁प्रेरणा ▁लेकर , ▁कुछ ▁अपनी ▁कल्पना ▁मिलाकर ▁‘ रक्त चरित्र ’ ▁को ▁दो ▁भागों ▁में ▁बनाया ▁है । ▁पहले ▁भाग ▁में ▁दिखाया ▁गया ▁है ▁कि ▁किस ▁तरह ▁अपने ▁पिता ▁और ▁भाई ▁की ▁हत्या ▁होने ▁के ▁बाद ▁रवि

In [56]:
learn.unfreeze()
learn.fit_one_cycle(1, slice(1e-3/(2.6**4),1e-3), moms=(0.8,0.7))

epoch,train_loss,valid_loss,matthews_correff,accuracy,time
0,0.463714,0.837389,0.435306,0.629032,00:16


In [57]:
learn.save('final')

In [58]:
from sklearn.metrics import accuracy_score, matthews_corrcoef
df_dict = {'query': list(df_test[1]), 'actual_label': list(df_test[0]), 'predicted_label': ['']*df_test.shape[0]}
all_nodes = list(set(df_train[0]))
for node in all_nodes:
    df_dict[node] = ['']*df_test.shape[0]
    
i2c = {}
for key, value in learn.data.c2i.items():
    i2c[value] = key
    
df_result = pd.DataFrame(df_dict)
for index, row in df_result.iterrows():
    pred = learn.predict(data_clas.test_ds[index])
    for node in all_nodes:
        row[node] = pred[2][learn.data.c2i[node]].item()
    row['predicted_label'] = i2c[pred[0].data.item()]
df_result.head()

,query,actual_label,predicted_label,neutral,positive,negative
0,काव्या अपनी खुन्नस और जिद में सिद्धार्थ को तबा...,negative,negative,0.132478,0.274723,0.5928
1,"निर्माता :\nफरहान अख्तर, रितेश सिधवानी\n\nनिर्...",negative,negative,0.170404,0.0415344,0.788062
2,असमर्पित रिश्ते में यकीन रखने वाले दोनों मौज-म...,positive,neutral,0.585512,0.181657,0.232831
3,"बेवकूफियां को हबीब फैज़ल ने लिखा है, जिनका नाम...",neutral,negative,0.117852,0.194204,0.687944
4,तब वह मोटी थी और जेरी ने उस पर ध्यान नहीं दिय...,neutral,neutral,0.500424,0.272833,0.226743


In [59]:
accuracy_score(df_result['actual_label'], df_result['predicted_label'])

0.5774193548387097

In [60]:
matthews_corrcoef(df_result['actual_label'], df_result['predicted_label'])

0.3723360207127859

In [61]:
df_result.to_csv('iitp+movie_result.csv', index=False)